# Data Exploration

## Install Packages

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as datetime
import json # The data has json file
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import gc
import plotly # 
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from flask import request
from datetime import datetime

## Run data

#### As some of the columns has json data type. We use json package to convert 

In [3]:
json_data=["device","geoNetwork","totals","trafficSource"]

gc.enable()

features = ['channelGrouping', 'date', 'fullVisitorId', 'visitId',\
       'visitNumber', 'visitStartTime', 'device_browser',\
       'device_deviceCategory', 'device_isMobile', 'device_operatingSystem',\
       'geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country',\
       'geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region',\
       'geoNetwork_subContinent', 'totals_bounces', 'totals_hits',\
       'totals_newVisits', 'totals_pageviews', 'totals_transactionRevenue',\
       'trafficSource_adContent', 'trafficSource_campaign',\
       'trafficSource_isTrueDirect', 'trafficSource_keyword',\
       'trafficSource_medium', 'trafficSource_referralPath',\
       'trafficSource_source','customDimensions']

def load_df(csv_path):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    ans = pd.DataFrame()
    dfs = pd.read_csv(csv_path, sep=',',
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                    chunksize = 100000)
    for df in dfs:
        df.reset_index(drop = True,inplace = True)
        for column in JSON_COLUMNS:
            column_as_df = json_normalize(df[column])
            column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
            df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
        use_df = df[features]
        del df
        gc.collect()
        ans = pd.concat([ans, use_df], axis = 0).reset_index(drop = True)
        print(ans.shape)
    return ans



In [4]:
train = load_df('Desktop/train_v2.csv')

(100000, 30)
(200000, 30)
(300000, 30)
(400000, 30)
(500000, 30)
(600000, 30)
(700000, 30)
(800000, 30)
(900000, 30)
(1000000, 30)
(1100000, 30)
(1200000, 30)
(1300000, 30)
(1400000, 30)
(1500000, 30)
(1600000, 30)
(1700000, 30)
(1708337, 30)


In [5]:
train.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,...,totals_pageviews,totals_transactionRevenue,trafficSource_adContent,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,customDimensions
0,Organic Search,20171016,3162355547410993243,1508198450,1,1508198450,Firefox,desktop,False,Windows,...,1,NaN,NaN,(not set),NaN,water bottle,organic,NaN,google,"[{'index': '4', 'value': 'EMEA'}]"
1,Referral,20171016,8934116514970143966,1508176307,6,1508176307,Chrome,desktop,False,Chrome OS,...,2,NaN,NaN,(not set),NaN,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com,"[{'index': '4', 'value': 'North America'}]"
2,Direct,20171016,7992466427990357681,1508201613,1,1508201613,Chrome,mobile,True,Android,...,2,NaN,NaN,(not set),True,NaN,(none),NaN,(direct),"[{'index': '4', 'value': 'North America'}]"
3,Organic Search,20171016,9075655783635761930,1508169851,1,1508169851,Chrome,desktop,False,Windows,...,2,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google,"[{'index': '4', 'value': 'EMEA'}]"
4,Organic Search,20171016,6960673291025684308,1508190552,1,1508190552,Chrome,desktop,False,Windows,...,2,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google,"[{'index': '4', 'value': 'Central America'}]"


In [6]:
train.totals_hits.describe()

count     1708337
unique        297
top             1
freq       864064
Name: totals_hits, dtype: object

# Deal with missing value 

The rest of column has missing value. 


In [9]:
def missing_values(data):
    total = data.isnull().sum().sort_values(ascending = False) # getting the sum of null values and ordering
    percent = (data.isnull().sum() / data.isnull().count() * 100 ).sort_values(ascending = False) #getting the percent and order of null
    df = pd.concat([total, percent], axis=1, keys=['Total', 'Percent']) # Concatenating the total and percent
    print("Total columns at least one Values: ")
    print (df[df['Total'] != 0]) # Returning values of nulls different of 0
    return 
missing_values(train) 

KeyboardInterrupt: 

In [8]:
#Deal with missing data 

#fill in NULL transaction revenue with zero 
train["totals_pageviews"]= train["totals_pageviews"].fillna(0).astype(int)
# Replace NA new visits with 0
train["totals_newVisits"]=train["totals_newVisits"].fillna(0).astype(int)
#Replace NA bounce with 0
train["totals_bounces"]=train["totals_bounces"].fillna(0).astype(int)
#Revenue Unknown replace with 0
train["totals_transactionRevenue"] = train["totals_transactionRevenue"].fillna(0.0).astype(float)
# fillna object feature
for col in ['trafficSource_keyword',
            'trafficSource_referralPath',
            'trafficSource_adContent']:
    train[col].fillna('unknown', inplace=True)

# fillna boolean feature
train['trafficSource_isTrueDirect'].fillna(False, inplace=True)


# Drop constant column

In [7]:
constant_column = [col for col in train.columns if train[col].nunique() == 1]
for c in constant_column:
    print(c + ':', train[c].unique())

print('drop columns:', constant_column)
train.drop(constant_column, axis=1, inplace=True)

drop columns: []


# Custom Dimensions

In [10]:
train['customDimensions'].unique()

array(["[{'index': '4', 'value': 'EMEA'}]",
       "[{'index': '4', 'value': 'North America'}]",
       "[{'index': '4', 'value': 'Central America'}]", '[]',
       "[{'index': '4', 'value': 'APAC'}]",
       "[{'index': '4', 'value': 'South America'}]"], dtype=object)

# Data Plot

### Device VS transactions

In [11]:
def chats(data):
    trace = go.Bar(y=data.index[::-1],
                   x=data.values[::-1],
                   showlegend=False,
                   orientation = 'h',
    )
    return trace

data=train.groupby("device_browser")["totals_transactionRevenue"].agg(["max","min","mean"])
data.columns=["maximum", "minimum", "mean"]
data=data.sort_values(by="mean",ascending=False)
trace1=chats(data["maximum"].head(10))
trace2=chats(data["minimum"].head(10))
trace3=chats(data["mean"].head(10))


data=train.groupby("device_deviceCategory")["totals_transactionRevenue"].agg(["max","min","mean"])
data.columns=["maximum", "minimum", "mean"]
data=data.sort_values(by="maximum",ascending=False)
trace4=chats(data["maximum"].head(10))
trace5=chats(data["minimum"].head(10))
trace6=chats(data["mean"].head(10))


data=train.groupby("device_operatingSystem")["totals_transactionRevenue"].agg(["size","count","mean"])
data.columns=["count", "count of non-zero revenue", "mean"]
data=data.sort_values(by="count",ascending=False)
trace7=chats(data["count"].head(10))
trace8=chats(data["count of non-zero revenue"].head(10))
trace9=chats(data["mean"].head(10))

In [12]:
fig = tools.make_subplots(rows=3, cols=3, vertical_spacing=0.04, 
                          subplot_titles=["device_browser - Count", "device_browser - Non-zero Revenue Count", "device_browser - Mean Revenue",
                                          "Device Category - Count",  "Device Category - Non-zero Revenue Count", "Device Category - Mean Revenue", 
                                          "Device OS - Count", "Device OS - Non-zero Revenue Count", "Device OS - Mean Revenue"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 2, 3)
fig.append_trace(trace7, 3, 1)
fig.append_trace(trace8, 3, 2)
fig.append_trace(trace9, 3, 3)

fig['layout'].update(height=1200, width=1200, paper_bgcolor='rgb(233,233,233)', title="Device Plots")
plotly.offline.plot(fig, filename='device-plots.html')


This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]
[ (2,1) x4,y4 ]  [ (2,2) x5,y5 ]  [ (2,3) x6,y6 ]
[ (3,1) x7,y7 ]  [ (3,2) x8,y8 ]  [ (3,3) x9,y9 ]



'device-plots.html'

### Conclusion 
<font color= darkblue>
    
1. Chrome provides the highest non-zero recenue, however customers who uses Firefox has the highest mean revenue
    
2. Customers who uses desktop will more willing to purchase more products in google shopping

3. Window user tends to purchase but chrome OS user will buy more items than windows users

### Geographic Info VS Transactions

In [14]:
def chats(data):
    trace = go.Bar(y=data.index[::-1],
                   x=data.values[::-1],
                   showlegend=False,
                   orientation = 'h',
    )
    return trace
data=train.groupby("geoNetwork_city")["totals_transactionRevenue"].agg(["size","count","mean"])
data.columns=["count", "count of non-zero revenue", "mean"]
data=data.sort_values(by="count",ascending=False)
trace1=chats(data["count"].head(10))
trace2=chats(data["count of non-zero revenue"].head(10))
trace3=chats(data["mean"].head(10))


data=train.groupby("geoNetwork_continent")["totals_transactionRevenue"].agg(["size","count","mean"])
data.columns=["count", "count of non-zero revenue", "mean"]
data=data.sort_values(by="count",ascending=False)
trace4=chats(data["count"].head(10))
trace5=chats(data["count of non-zero revenue"].head(10))
trace6=chats(data["mean"].head(10))


data=train.groupby("geoNetwork_country")["totals_transactionRevenue"].agg(["size","count","mean"])
data.columns=["count", "count of non-zero revenue", "mean"]
data=data.sort_values(by="count",ascending=False)
trace7=chats(data["count"].head(10))
trace8=chats(data["count of non-zero revenue"].head(10))
trace9=chats(data["mean"].head(10))


# Creating two subplots
fig = tools.make_subplots(rows=3, cols=3, vertical_spacing=0.04, 
                          subplot_titles=["geoNetwork_city - Count", "geoNetwork_city - Non-zero Revenue Count", "geoNetwork_city- Mean Revenue",
                                          "geoNetwork_continent - Count",  "geoNetwork_continent - Non-zero Revenue Count", "geoNetwork_continent - Mean Revenue", 
                                          "geoNetwork_country - Count", "geoNetwork_country - Non-zero Revenue Count", "geoNetwork_country - Mean Revenue"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 2, 3)
fig.append_trace(trace7, 3, 1)
fig.append_trace(trace8, 3, 2)
fig.append_trace(trace9, 3, 3)

fig['layout'].update(height=1200, width=1200, paper_bgcolor='rgb(233,233,233)', title="Geographical Plots")
plotly.offline.plot(fig, filename='Geographical.html')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]
[ (2,1) x4,y4 ]  [ (2,2) x5,y5 ]  [ (2,3) x6,y6 ]
[ (3,1) x7,y7 ]  [ (3,2) x8,y8 ]  [ (3,3) x9,y9 ]



'Geographical.html'


### Conclusion 
<font color=darkblue>
1. Customers in NEW YORK AND SF are the main revenue stream 
  
2. US generates the most revenue but Asia has relative large purchase population

3. Canada also has relative high revenue

In [15]:
def chats(data):
    trace = go.Scatter(x=data.index[::-1],
                   y=data.values[::-1],
                   mode='markers'
    )
    return trace


data=train.groupby("totals_hits")["totals_transactionRevenue"].agg(["size","count","mean"])
data.columns=["count", "count of non-zero revenue", "mean"]
trace1=chats(data["count"])
trace2=chats(data["count of non-zero revenue"])
trace3=chats(data["mean"])



In [36]:

data=train.groupby("totals_pageviews")["totals_transactionRevenue"].agg(["count","mean"])
data.columns=[ "count of non-zero revenue", "mean"]
trace5=chats(data["count of non-zero revenue"])
trace6=chats(data["mean"])
# Creating two subplots
fig = tools.make_subplots(rows=2, cols=3, vertical_spacing=0.04, 
                          subplot_titles=[
                                            "totals_hits - Non-zero Revenue Count", "totals_hits - Mean Revenue", 
                                          "totals_pageviews - Count", "totals_pageviews - Non-zero Revenue  Count", "totals_pageviews - Mean Revenue"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 2, 3)

fig['layout'].update(height=1200, width=1200, paper_bgcolor='rgb(233,233,233)', title="Webpage Performance Plots")
plotly.offline.plot(fig, filename='Webpage Performance.html')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]
[ (2,1) x4,y4 ]  [ (2,2) x5,y5 ]  [ (2,3) x6,y6 ]



'Webpage Performance.html'

### Time Series Revenue Performance

In [8]:
format_str = '%Y%m%d'
train['formated_date'] = train['date'].apply(lambda x: datetime.strptime(str(x), format_str))
train['_year'] = train['formated_date'].apply(lambda x:x.year)
train['_month'] = train['formated_date'].apply(lambda x:x.month)
train['_quarterMonth'] = train['formated_date'].apply(lambda x:x.day//8)
train['_day'] = train['formated_date'].apply(lambda x:x.day)
train['_weekday'] = train['formated_date'].apply(lambda x:x.weekday())

In [42]:
data=train.groupby("_weekday")["totals_transactionRevenue"].agg("sum")
data.to_frame()

,totals_transactionRevenue
_weekday,
0,3.936477e+11
1,4.404847e+11
2,5.038629e+11
3,3.951166e+11
4,3.826879e+11
5,8.993320e+10
6,1.106120e+11


In [43]:
data=train.groupby("_month")["totals_transactionRevenue"].agg("mean")
data.to_frame()

,totals_transactionRevenue
_month,
1,1.149448e+06
2,1.196654e+06
3,1.343003e+06
4,1.591050e+06
5,1.861857e+06
6,1.912131e+06
7,1.733684e+06
8,1.947741e+06
9,1.449313e+06


People generally purchase more during summer time

In [11]:
data=train.groupby("_month")["totals_transactionRevenue"].agg("mean")
data.columns=["mean"]
trace3=chats(data)

NameError: name 'chats' is not defined

In [ ]:
trace3

In [46]:
data=train.groupby("_quarterMonth")["totals_transactionRevenue"].agg("mean")
data.to_frame()

,totals_transactionRevenue
_quarterMonth,
0,1.336164e+06
1,1.411481e+06
2,1.320793e+06
3,1.351747e+06


In [52]:
data=train.groupby("visitId")["totals_transactionRevenue"].agg("sum").to_frame()
data_pos=data[data.totals_transactionRevenue>0]
data_pos

,totals_transactionRevenue
visitId,
1470046365,1.589400e+09
1470050783,8.990000e+06
1470052998,2.719000e+07
1470055898,7.170000e+07
1470057237,1.674000e+08
1470061745,1.278000e+08
1470063041,2.348000e+07
1470063685,3.996000e+07
1470064251,2.211380e+09


## Select Features

In [12]:
null_cnt = train.isnull().sum().sort_values()
print(null_cnt[null_cnt > 0])

Series([], dtype: int64)


In [13]:
train.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,...,trafficSource_medium,trafficSource_referralPath,trafficSource_source,customDimensions,formated_date,_year,_month,_quarterMonth,_day,_weekday
0,Organic Search,20171016,3162355547410993243,1508198450,1,1508198450,Firefox,desktop,False,Windows,...,organic,unknown,google,"[{'index': '4', 'value': 'EMEA'}]",2017-10-16,2017,10,2,16,0
1,Referral,20171016,8934116514970143966,1508176307,6,1508176307,Chrome,desktop,False,Chrome OS,...,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com,"[{'index': '4', 'value': 'North America'}]",2017-10-16,2017,10,2,16,0
2,Direct,20171016,7992466427990357681,1508201613,1,1508201613,Chrome,mobile,True,Android,...,(none),unknown,(direct),"[{'index': '4', 'value': 'North America'}]",2017-10-16,2017,10,2,16,0
3,Organic Search,20171016,9075655783635761930,1508169851,1,1508169851,Chrome,desktop,False,Windows,...,organic,unknown,google,"[{'index': '4', 'value': 'EMEA'}]",2017-10-16,2017,10,2,16,0
4,Organic Search,20171016,6960673291025684308,1508190552,1,1508190552,Chrome,desktop,False,Windows,...,organic,unknown,google,"[{'index': '4', 'value': 'Central America'}]",2017-10-16,2017,10,2,16,0


In [14]:
train.drop(['visitId','visitStartTime','date'],axis=1,inplace=True)
for i, t in train.loc[:, train.columns != 'fullVisitorId'].dtypes.iteritems():
    if t == object:
        train[i].fillna('unknown', inplace=True)
        train[i] = pd.factorize(train[i])[0]

In [15]:
train.head()

,channelGrouping,fullVisitorId,visitNumber,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,geoNetwork_city,geoNetwork_continent,geoNetwork_country,...,trafficSource_medium,trafficSource_referralPath,trafficSource_source,customDimensions,formated_date,_year,_month,_quarterMonth,_day,_weekday
0,0,3162355547410993243,1,0,0,False,0,0,0,0,...,0,0,0,0,2017-10-16,2017,10,2,16,0
1,1,8934116514970143966,6,1,0,False,1,1,1,1,...,1,1,1,1,2017-10-16,2017,10,2,16,0
2,2,7992466427990357681,1,1,1,True,2,0,1,1,...,2,0,2,1,2017-10-16,2017,10,2,16,0
3,0,9075655783635761930,1,1,0,False,0,0,2,2,...,0,0,0,0,2017-10-16,2017,10,2,16,0
4,0,6960673291025684308,1,1,0,False,0,0,1,3,...,0,0,0,2,2017-10-16,2017,10,2,16,0
